# An interactive notebook for exploring the model and trading strategy

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from model_functions import create_features, train_model, simulate_trading, optimize_trading_strategy
from kraken_functions import download_full_ohlc_data
from sklearn.metrics import mean_squared_error

# Configuration parameters
PAIR = 'XXRPZEUR'
INTERVAL = 60  # minutes
TRAIN_SPLIT = 0.7  # 70% for training
TEST_SPLIT = 0.8   # 80% total for train+val, rest for test
INITIAL_BALANCE = 100.0  # EUR
MIN_CONFIDENCE = 0.02  # 2% minimum return threshold
POSITION_SIZE = 25  # EUR position size

# Load and prepare data
df = download_full_ohlc_data(pair=PAIR, interval=INTERVAL)
df = create_features(df)
df = df.dropna(subset=['target'])
X = df.drop(columns=['timestamp', 'target'])
y = df['target']

# Split data
test_split_idx = int(len(df) * TEST_SPLIT)
val_split_idx = int(len(df) * TRAIN_SPLIT)

X_train = X[:val_split_idx]
X_val = X[val_split_idx:test_split_idx]
X_test = X[test_split_idx:]

y_train = y[:val_split_idx]
y_val = y[val_split_idx:test_split_idx]
y_test = y[test_split_idx:]

# Train model and make predictions
final_model = train_model(X_train, X_val, y_train, y_val)
y_train_pred = final_model.predict(X_train)
y_val_pred = final_model.predict(X_val)
y_test_pred = final_model.predict(X_test)

# Calculate MSE for each set
train_mse = mean_squared_error(y_train, y_train_pred)
val_mse = mean_squared_error(y_val, y_val_pred)
test_mse = mean_squared_error(y_test.dropna(), y_test_pred[y_test.notna()])

print(f"Initial Model MSE Scores:")
print(f"Training: {train_mse:.6f}")
print(f"Validation: {val_mse:.6f}")
print(f"Test: {test_mse:.6f}")

# Optimize trading strategy
val_df = df.iloc[val_split_idx:test_split_idx].copy()
best_params, best_balance = optimize_trading_strategy(val_df, y_val_pred)

print("\nBest Parameters:", best_params)
print("Best Final Balance on validation set:", best_balance)

# Apply optimized strategy to test set
test_df = df[test_split_idx:].copy()
test_df['predicted_price'] = y_test_pred
final_balance, trades = simulate_trading(
    test_df,
    predicted_price_col="predicted_price",
    **best_params,
    initial_balance=INITIAL_BALANCE
)

# Print trading results
total_return = ((final_balance - INITIAL_BALANCE) / INITIAL_BALANCE) * 100
test_results = f"""Test Set Results:
Initial Balance: {INITIAL_BALANCE:.2f} EUR
Final Balance: {final_balance:.2f} EUR
Total Return: {total_return:.2f}%

Trade History:
"""

# Print trade details
for i in range(0, len(trades), 2):
    buy = trades[i]
    sell = trades[i+1] if i+1 < len(trades) else None
    
    if sell:
        profit_pct = (sell['price'] - buy['price']) / buy['price'] * 100
        
        # Convert timestamps to pandas datetime objects
        buy_time = pd.to_datetime(buy['timestamp'])
        sell_time = pd.to_datetime(sell['timestamp'])
        
        test_results += f"\nTrade {i//2 + 1}:\n"
        test_results += f"Buy:  {buy_time.strftime('%m-%d %H:%M')} @ {buy['price']:.5f} EUR\n"
        test_results += f"Sell: {sell_time.strftime('%m-%d %H:%M')} @ {sell['price']:.5f} EUR ({profit_pct:.2f}%)"

print(test_results)

# Visualize predictions
plt.figure(figsize=(15, 6))

train_dates = df['timestamp'][:val_split_idx]
val_dates = df['timestamp'][val_split_idx:test_split_idx]
test_dates = df['timestamp'][test_split_idx:]

plt.plot(df['timestamp'], df['target'], 'b-', alpha=0.5, label='Actual')
plt.plot(train_dates, y_train_pred, 'r--', label='Train Predictions')
plt.plot(val_dates, y_val_pred, 'g--', label='Validation Predictions')
plt.plot(test_dates, y_test_pred, 'y--', label='Test Predictions')

plt.title('Initial Model: Price Predictions vs Actual Values')
plt.xlabel('Date')
plt.ylabel('Price (EUR)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('initial_model_predictions.png')
plt.close()

final_model = train_model(X_train, X_val, y_train, y_val, X_test=X_test, y_test=y_test)

Downloaded data starting from 2024-12-07 15:00:00
Initial Model MSE Scores:
Training: 0.000019
Validation: 0.003418
Test: 0.008296

Best Parameters: {'buy_threshold': 4.393974579690156, 'take_profit_threshold': 1.7377800944726811, 'max_hold_hours': 27}
Best Final Balance on validation set: 101.43490448062968
Test Set Results:
Initial Balance: 100.00 EUR
Final Balance: 117.47 EUR
Total Return: 17.47%

Trade History:

Trade 1:
Buy:  01-01 00:00 @ 2.01556 EUR
Sell: 01-01 00:00 @ 1.93881 EUR (-3.81%)
Trade 2:
Buy:  01-01 00:00 @ 1.93780 EUR
Sell: 01-01 00:00 @ 1.98212 EUR (2.29%)
Trade 3:
Buy:  01-01 00:00 @ 1.98527 EUR
Sell: 01-01 00:00 @ 2.04259 EUR (2.89%)
Trade 4:
Buy:  01-01 00:00 @ 2.05205 EUR
Sell: 01-01 00:00 @ 2.09138 EUR (1.92%)
Trade 5:
Buy:  01-01 00:00 @ 2.08767 EUR
Sell: 01-01 00:00 @ 2.15013 EUR (2.99%)
Trade 6:
Buy:  01-01 00:00 @ 2.18382 EUR
Sell: 01-01 00:00 @ 2.23804 EUR (2.48%)
Trade 7:
Buy:  01-01 00:00 @ 2.23684 EUR
Sell: 01-01 00:00 @ 2.30140 EUR (2.89%)
Trade 8:
Buy